In [ ]:
import yaml
with open("sources.yml", 'r') as f:
    yaml_data = yaml.load(f, Loader=yaml.FullLoader)

print(yaml_data["airports"][2018])



In [ ]:
def create_data_list(source_file):
    with open(source_file, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    return(yaml_data)


yaml_data = create_data_list("sources.yml")

In [ ]:
from src.create_data_list import create_data_list
urls = create_data_list("sources.yml")

In [ ]:
import pandas as pd

data = pd.read_csv(
    yaml_data["airports"][2018], 
    delimiter = ';',
    dtype = {
        "ANMOIS": "str",  
        "APT": "str",     
        "APT_NOM": "str", 
        "APT_ZON": "str",
    }
)

data["an"] = data["ANMOIS"].str.slice(stop=4)
data["mois"] = data["ANMOIS"].str.slice(start=4).str.replace('^0', '', regex=True)
data.head()

In [ ]:
def clean_dataframe(df):
    df["an"]   = df["ANMOIS"].str.slice(stop=4)
    df["mois"] = df["ANMOIS"].str.slice(start=4).str.replace('^0', '', regex=True)

    df.columns = [x.lower() for x in df.columns]

    return(df)

clean_dataframe(data)